In [1]:
import torch
import math
from torch import nn, autograd, stack, cat
from torch.autograd import Variable
import torch.utils.model_zoo as model_zoo
import torch.nn.functional as F

In [ ]:
class ViewPooler(nn.Module):
    def __init__(self, in_features, out_features, features_extractor, hidden, views):
        super(ViewPooler, self).__init__()
        self.features_extractor = features_extractor
        self.view_pooling = nn.MaxPool2d(kernel_size=(views,1))
        self.relu = nn.ReLU()
        self.fc_1 = nn.Linear(in_features, hidden)
        self.fc_2 = nn.Linear(hidden,out_features)
        self.dropout = nn.Dropout(p=0.3)
        self.bn = nn.BatchNorm1d(in_features, momentum=0.01)
        self.intermediate = intermediate

    def forward(self, views):
        views_features = [self.features_extractor(view) for view in views]
        
        ims,filters,h,w = views_features[0].shape
        
        flat_views = [view.view(ims,-1) for view in views_features]
        stacked_views = torch.stack(flat_views,dim=1)
        pooled_views = self.view_pooling(stacked_views).squeeze()
        
        features = self.bn(pooled_views)
        features = self.fc_1(features)
        features = self.relu(features)
        features = self.dropout(features)
        features = self.fc_2(features)
        
        return features

In [5]:
class Old_ViewPooler(nn.Module):
    def __init__(self, in_features, out_features, features_extractor, views):
        super(Old_ViewPooler, self).__init__()
        #self.in_features = in_features
        self.features_extractor = features_extractor
        self.view_pooling = nn.MaxPool2d(kernel_size=(views,1))
        self.relu = nn.ReLU()
        self.fc_1 = nn.Linear(in_features, 100)
        self.fc_2 = nn.Linear(100,out_features)
        self.dropout = nn.Dropout(p=0.2)
        self.bn = nn.BatchNorm1d(in_features, momentum=0.01)

    def forward(self, views):
        views_features = [self.features_extractor(view) for view in views]
        
        ims,filters,h,w = views_features[0].shape
        
        flat_views = [view.view(ims,-1) for view in views_features]
        stacked_views = stack(flat_views,dim=1)
        pooled_views = self.view_pooling(stacked_views).squeeze()
        
        features = self.bn(pooled_views)
        features = self.fc_1(features)
        features = self.relu(features)
        features = self.dropout(features)
        features = self.fc_2(features)
        
        return features
